In [1]:
from task import *

/data/mn27889/miniconda3/envs/mental-health-agents/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


path is  /data/mn27889/mental-health-agents/src/sdoh_analysis/helpers/../models/setfit-mental-health-topic-prediction/setfit-small-research-topic-prediction/content/setfit-small-research-topic-prediction
True


/data/mn27889/miniconda3/envs/mental-health-agents/lib/python3.12/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/data/mn27889/miniconda3/envs/mental-health-agents/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  w

encoding database


2775it [00:00, 437892.91it/s]


got 2511 of 2749 others out bc of size


100%|██████████| 883/883 [00:00<00:00, 294284.50it/s]

encodnig databsae - this can take a while - TODO - cache this


db encoded


In [19]:
import re

In [20]:
task = MentalHealthTask()

Repo card metadata block was not found. Setting CardData to empty.


In [21]:
ProfoundExpression_id = 3
Specialist_Minimum_9_id = 7
cosmic_love_28_id = 8
idx = cosmic_love_28_id
x = task.get_input(idx)

In [22]:
print(x)

(22f) I’ve been struggling with depression since I was a young child (my earliest memory of being depressed was when I was around 4 or 5 years old). I’m now at a point in my life where I should be happy, I have a nice car, a good job with colleagues who respect me and with whom I get along well, I’m doing well in school and I have good friends who care about me, my relationship with my siblings is also great. I don’t have to worry about rent or living expenses beyond gas for my car, I don’t have student loans bc I have scholarships that pay for my tuition and books. My physical health is good, I’ve been eating healthy and exercising at least 4 times per week. I have it all but I still feel like I have nothing. I still feel so empty and I’m lonely even when I’m surrounded by friends and family. I’m touch starved yet the thought of having someone touch me makes me nauseous, and I don even know how I’d ask for a hug even if I wanted one. I experienced physical, emotional and verbal abuse 

In [23]:
determinant_prompt, example_format, sdoh_prompt = task.get_cot_sdoh_prompt_data(input=x, idx=idx)

In [24]:
final_prompt = user_prompt_persona.format(user_input=x, user_past_history=sdoh_prompt)

In [25]:
print(final_prompt)

You are given a user input shared by a mentally ill person along with the past history of the same person categorized by social determinants.
Considering the user input and the past history of the user, your job is to determine 3 therapy techniques out of the following 8 therapy techniques
to better improve the overall mental health of the person: (1) Cognitive Behavioral Therapy (2) Exposure Therapy (3) Psychodynamic Therapy (4) Client Centered Therapy
(5) Humanistic Therapy (6) Interpersonal Therapy (7) Mentalization Therapy (8) Mindfulness Therapy.

User Input: (22f) I’ve been struggling with depression since I was a young child (my earliest memory of being depressed was when I was around 4 or 5 years old). I’m now at a point in my life where I should be happy, I have a nice car, a good job with colleagues who respect me and with whom I get along well, I’m doing well in school and I have good friends who care about me, my relationship with my siblings is also great. I don’t have to 

In [26]:
output = gpt(user_prompt=final_prompt, system_prompt=sys_prompt_persona)
answer = output[0]
print(answer)

Therapy Technique: Cognitive Behavioral Therapy | Interpersonal Therapy | Psychodynamic Therapy


In [27]:
match = re.match(r".*Therapy Technique:([\s\|\w]+).*", answer, re.DOTALL)

In [28]:
therapy_techniques = match.groups()[0]
therapy_list = therapy_techniques.split('|')
therapy_list = [therapy.strip() for therapy in therapy_list]

In [29]:
final_counsel = []
for therapy in therapy_list:
    final_prompt_therapy = user_prompt_therapy.format(therapy_technique=therapy, user_input=x, user_past_history=sdoh_prompt)
    therapy_response = gpt(user_prompt=final_prompt_therapy, temperature=0.7, n=3)

    extracted_counsel = []
    for index, out in enumerate(therapy_response):
        match_counsel = re.match(r".*Therapy Counsel.*:\n*(.*)", out, re.DOTALL)
        extracted_counsel.append(match_counsel.groups()[0])
    
    vote_choice = ""
    for index, counsel in enumerate(extracted_counsel):
        vote_choice += "Choice " + str(index+1) + ":\n\n" + counsel + "\n\n"
        
    final_prompt_vote = user_vote_prompt.format(user_input=x, user_past_history=sdoh_prompt, counselling_choices=vote_choice)
    vote_score = gpt(user_prompt=final_prompt_vote, system_prompt=system_vote_prompt, temperature=0.7, n=1)
    print(vote_score)
    match_vote_score = re.match(r".*best choice is .*(\d+).*", vote_score[0], re.DOTALL)
    
    print(extracted_counsel[int(match_vote_score.groups()[0])-1])
    final_counsel.append(extracted_counsel[int(match_vote_score.groups()[0])-1])

["Analysis of Choices: \n\nChoice 1:\n\n1. Medium Sensitivity (MS): 8 - The message is delivered with sensitivity, acknowledging the user's feelings and experiences.\n2. Hope and Positive Expectations (HPE): 7 - The message provides hope by mentioning strategies to address loneliness and manage depression. \n3. Persuasiveness (PER): 7 - The message is persuasive in encouraging the user to work on communication and connection.\n4. Emotional Engagement (EEn): 8 - The message engages with the user's emotions and validates them.\n5. Warmth, Acceptance & Understanding (WAU): 8 - The message expresses warmth and understanding towards the user's situation.\n6. Empathy (EMP): 8 - The message empathizes with the user's struggles regarding loneliness and trauma.\n7. Alliance-Bond Capacity (ABC): 8 - The message builds an alliance with the user by offering to work together on strategies.\n8. Alliance Rupture Repair Responsiveness (ARRR): 7 - The message acknowledges the user's loss and offers to 

In [30]:
final_counsel

['I understand that you\'re feeling very low right now, and I appreciate your courage in sharing your feelings. It sounds like despite having stability and support in certain areas of your life, you\'re still dealing with a deep-seated sadness that has been with you since childhood. Your feelings are valid and important. \n\nFirstly, it\'s crucial to acknowledge the impact of the abuse you experienced growing up. Trauma can deeply affect how we perceive ourselves and the world around us. It\'s okay that you\'re struggling with this, and it doesn\'t make your achievements any less significant. \n\nYou\'ve mentioned feeling lonely even when surrounded by friends and family. Loneliness can sometimes be a sense of not feeling understood or connected, even when people are around. We can work on strategies to address this, such as improving communication with your loved ones and exploring new ways to connect with people.\n\nYou\'ve also mentioned feeling "touch starved," which could be indic

In [31]:
counsellor_advice_choices = ""
for index, advice in enumerate(final_counsel):
    counsellor_advice_choices += "Advice " + str(index+1) + ":\n\n" + advice + "\n\n"

final_prompt_summarize = user_summarize_prompt.format(user_input=x, user_past_history=sdoh_prompt, counselling_advices=counsellor_advice_choices)
summarized_response = gpt(user_prompt=final_prompt_summarize, system_prompt=system_summarize_prompt, temperature=0.7, n=1)

In [32]:
print(summarized_response[0])

I understand you're going through an extremely challenging time, and it's commendable that you've shared your feelings. Despite the stability and apparent successes in your life, you're battling a deep-rooted depression that has been a part of you since your early childhood. This is a significant struggle, and your feelings are valid.

Firstly, it's crucial we acknowledge the impact of the abuse you experienced in your childhood. Such trauma can profoundly affect our self-perception and worldview. It's okay to be grappling with this, and it doesn't make your accomplishments any less noteworthy. We can work together to address these past experiences and initiate the healing process.

You've expressed feelings of loneliness despite being surrounded by supportive friends and family, which could indicate issues with social acceptance and connection. It's also evident that you crave physical touch, yet feel apprehensive due to your past experiences. We can work on improving your communicati

In [34]:
print(summarized_response[0].replace('\n\n', '\n'))

I understand you're going through an extremely challenging time, and it's commendable that you've shared your feelings. Despite the stability and apparent successes in your life, you're battling a deep-rooted depression that has been a part of you since your early childhood. This is a significant struggle, and your feelings are valid.
Firstly, it's crucial we acknowledge the impact of the abuse you experienced in your childhood. Such trauma can profoundly affect our self-perception and worldview. It's okay to be grappling with this, and it doesn't make your accomplishments any less noteworthy. We can work together to address these past experiences and initiate the healing process.
You've expressed feelings of loneliness despite being surrounded by supportive friends and family, which could indicate issues with social acceptance and connection. It's also evident that you crave physical touch, yet feel apprehensive due to your past experiences. We can work on improving your communication

### Evaluating the response using Psychotherapy Factors

In [ ]:
cognitive_behavior = '''
I understand that you're going through a lot right now. It seems like you're struggling with feelings of self-doubt, guilt, and fear of social situations. It's okay to feel this way, and I want to assure you that you're not alone.
Firstly, it's important to understand that everyone has their unique pace and way of experiencing emotions, and it's okay if yours are different from others. It doesn't mean that you're less real or less human. Your feelings and experiences are valid, and it's okay to express them.
You've mentioned that you constantly seek approval from others, which tells me that you might be doubting your self-worth. It's essential to understand that your value doesn't depend on others' approval. Engaging in activities that you enjoy can help boost your self-esteem and reduce these feelings over time.
You also seem to have some fears around social situations. It's okay to feel anxious, but remember that everyone makes mistakes in social settings, and it's a part of learning and growing. With practice, you can improve your social skills and feel more comfortable around others.
In terms of your work, it seems like you find it difficult to engage in tasks unless you're in 'work mode'. It might be helpful to explore different career options that align with your interests. 
You also mentioned a feeling of guilt about not doing enough. It's okay to feel this way, but remember that everyone has their limits. It's important to take care of yourself first before you can help others.
Lastly, remember that it's okay to seek help. You're taking the first step by reaching out and trying to understand your feelings. It's a journey, and I'm here to support you through it.
'''

In [ ]:
final_prompt_evaluation = user_evaluation_prompt.format(counselling_advice=cognitive_behavior)
evaluation_output = gpt(user_prompt=final_prompt_evaluation, system_prompt=system_evaluation_prompt, temperature=0.7, n=1)

In [ ]:
print(evaluation_output[0])

In [ ]:
interpersonal_therapy = '''
Thank you for sharing your feelings and thoughts. I want to assure you that it's okay to feel this way and you're not alone. It seems like you're dealing with a lot internally, particularly with social anxiety, self-judgment, and a need for approval. 
In interpersonal therapy, we focus on the relationships that may be causing or worsening these feelings. It sounds like your relationship with your parents is one area we could explore. You've mentioned that you don't have a relationship with them, which can significantly impact how you form relationships with others and how you view yourself.
You also mentioned a desire for a partner but fear about ruining potential relationships. It's important to remember that everyone has fears about relationships, and it's okay to be cautious. We can work together to explore these feelings more and develop strategies to manage these fears.
In terms of interests, it's okay not to have many. However, exploring new hobbies or activities may help you find something you enjoy and can help distract you from constant self-evaluation and approval-seeking behaviors.
Lastly, you mentioned feeling guilty about not being a better person. Remember, everyone has their unique strengths and weaknesses, and it's unrealistic to expect oneself to save the world. Let's work on identifying small, realistic ways you can contribute positively to the world and alleviate some of this guilt.
Remember, it's okay to feel the way you do, and you're already taking a significant step by seeking help. We'll work together to improve your mental health and navigate through these feelings.
'''

In [ ]:
final_prompt_evaluation = user_evaluation_prompt.format(counselling_advice=interpersonal_therapy)
evaluation_output = gpt(user_prompt=final_prompt_evaluation, system_prompt=system_evaluation_prompt, temperature=0.7, n=1)

In [ ]:
print(evaluation_output[0])

In [ ]:
mindfulness_therapy = '''
I understand that you're feeling lost and overwhelmed right now. It's okay to feel this way, and I commend you for reaching out and sharing your feelings, which is the first step towards healing. 
From your past history and current feelings, it seems like you're struggling with self-acceptance and self-judgment. It's important to remember that every individual is unique. You don't have to compare yourself with others or seek their approval to feel validated. 
One thing that can help is mindfulness, focusing on the present moment without judgment. This could help reduce your feelings of fear and anxiety in social situations. There are many techniques to practice mindfulness, such as meditation, deep breathing, and yoga. 
It sounds like you may also be dealing with negative thoughts and beliefs about yourself. Cognitive Behavioral Therapy can be a helpful tool in identifying these thoughts and learning to challenge them.
You mentioned a difficulty in expressing your emotions. Art, music, writing, or even physical activities can be helpful outlets for emotions. They can help you understand and express your feelings in a non-verbal way.
You've expressed a desire for a romantic relationship and concerns about your physical appearance requirements. Remember, a healthy relationship is based on mutual respect, understanding, and love, not just physical attributes.
Lastly, although you feel like you don't have many interests, I encourage you to explore new activities or hobbies. Finding something you enjoy can bring a sense of achievement and happiness that doesn't rely on others' approval.
Remember, it's okay not to have it all figured out. Your feelings are valid, and it's important to give yourself permission to explore who you are and what you want from life at your own pace. It's okay to ask for help, and it's okay to take time for yourself. You're not alone in this journey.
'''

In [ ]:
final_prompt_evaluation = user_evaluation_prompt.format(counselling_advice=mindfulness_therapy)
evaluation_output = gpt(user_prompt=final_prompt_evaluation, system_prompt=system_evaluation_prompt, temperature=0.7, n=1)

In [ ]:
print(evaluation_output[0])

In [ ]:
summarized_therapy = '''
I'm sorry to hear you're feeling this way, but please know that it's okay to feel as you do and you're not alone. Your struggle with self-doubt, guilt, and fear of social situations is valid, and it's good that you're reaching out to understand your feelings better.
Based on the advice from multiple counsellors, it seems that you are having a hard time with self-acceptance and you often seek approval from others. Remember, everyone's emotional experience is unique and your feelings are as valid as anyone else's. It's important to understand that your worthiness is not contingent on others' approval. You might find it beneficial to engage in activities you enjoy, as doing so can help improve your self-esteem and shift these feelings over time.
You've expressed fear of social situations. Remember, it's okay to make mistakes in social interactions; they're part of the learning process. You can gradually build your social skills and confidence with practice. Mindfulness techniques, such as deep breathing and meditation, could also help in managing anxiety in these situations.
Your relationship with your parents, or the lack thereof, and your desire for a partner seem to be significant factors in your life. In therapy, we could explore how these relationships affect your self-perception and your interactions with others. It's okay to be careful about relationships, and we can work together on strategies to manage these fears.
You've mentioned your difficulty with work and feeling guilty about not doing enough. It's crucial to remember that we all have limits, and it's okay not to be able to do everything. Exploring career options that align with your interests might make work more enjoyable and less stressful. Also, it's unrealistic and overwhelming to attempt to "save the world." Instead, we can focus on identifying small, realistic ways for you to contribute positively to your environment.
In terms of expressing your emotions, consider creative outlets like art, music, or writing. These can provide a non-verbal way to understand and express your feelings. Moreover, it's okay not to have many interests; however, you're encouraged to explore new hobbies or activities. Discovering something you enjoy can give you a sense of achievement and joy that doesn't depend on others' approval.
Lastly, remember that a healthy relationship isn't just about physical attributes but mutual respect, understanding, and love. Your feelings are valid, and it's okay to explore who you are and what you want from life at your own pace. Don't hesitate to seek help when needed - you're not alone in this journey.
'''

In [ ]:
final_prompt_evaluation = user_evaluation_prompt.format(counselling_advice=summarized_therapy)
evaluation_output = gpt(user_prompt=final_prompt_evaluation, system_prompt=system_evaluation_prompt, temperature=0.7, n=1)

In [ ]:
print(evaluation_output[0])

### Testing Code

In [ ]:
final_prompt_therapy = user_prompt_therapy.format(therapy_technique=therapy_list[1], user_input=x, user_past_history=sdoh_prompt)

In [ ]:
print(final_prompt_therapy)

In [ ]:
output = gpt(user_prompt=final_prompt_therapy, temperature=0.7, n=3)

In [ ]:
output

In [ ]:
example_output = output

In [ ]:
extracted_counsel = []
for index, out in enumerate(example_output):
    match_counsel = re.match(r".*Therapy Counsel.*:\n*(.*)", out, re.DOTALL)
    extracted_counsel.append(match_counsel.groups()[0])

In [ ]:
vote_choice = ""
for index, out in enumerate(extracted_counsel):
    vote_choice += "Choice " + str(index+1) + ":\n\n" + out + "\n\n"

In [ ]:
print(vote_choice)

In [ ]:
final_prompt_vote = user_vote_prompt.format(user_input=x, user_past_history=sdoh_prompt, counselling_choices=vote_choice)

In [ ]:
output = gpt(user_prompt=final_prompt_vote, system_prompt=system_vote_prompt, temperature=0.7, n=1)

In [ ]:
output

In [ ]:
pattern = r".*best choice is .*(\d+).*"
match = re.match(pattern, output[0], re.DOTALL)

In [ ]:
sel_counsel = extracted_counsel[int(match.groups()[0])-1]

In [ ]:
print(sel_counsel)

In [ ]:
cognitive_response = '''
I understand that you're feeling lost, disconnected, and critical of yourself right now. It's important to remember that it's okay to feel these things, and it's okay to ask for help.

Firstly, let's work on building self-compassion. You're very hard on yourself, and it's crucial to learn to treat yourself with the same kindness you'd extend to others. There are exercises we can work through together to help you build this skill.

Secondly, your thought patterns seem to be dominated by negative perceptions of yourself. Cognitive Behavioral Therapy (CBT) can be very effective in identifying and challenging these thoughts, and we can use cognitive restructuring exercises to help you with this.

Social situations appear to cause you a lot of distress. We can gradually expose you to these situations in a controlled manner to help reduce your fear over time. Remember, it's okay to be yourself and not everyone needs to approve of you. Your worth is not defined by others' approval.

Your need for approval and your feelings about your physical appearance suggest low self-esteem. Building a positive self-image independent of others can be empowering and freeing. 

Your struggle with emotional expression is something we can address with emotional intelligence training. This will help you understand your emotions and express them appropriately.

You've mentioned that you have very few interests. Exploring different activities might help you find something you're passionate about, which can bring joy and purpose into your life.

Your difficulties with planning and losing things suggest that working on your organizational skills could be beneficial. This can help reduce anxiety and feelings of being overwhelmed.

Finally, incorporating mindfulness and relaxation techniques into your daily routine can help manage feelings of guilt and anxiety, and improve focus and observation skills. 

Remember, improvement takes time and patience. It's okay to take small steps. You're not alone in this journey, and I'm here to support you along the way.
'''

In [ ]:
interpersonal_therapy = '''
I understand that you're feeling lost and scared socially, and are struggling with your self-perception. It's okay to feel this way, and it's a step forward that you're acknowledging these feelings. In interpersonal therapy, we focus on the relationships that you have with others, including the one with yourself. 

It seems like you're seeking approval from others, but it's also important to seek approval from yourself. It's not narcissistic to want to feel good about yourself. Instead of focusing on how others perceive you, let's work on how you perceive yourself. 

Your relationship with your parents and your fear of falling in love seem to be significant factors in your life. Let's explore these feelings in more depth, and understand how they're affecting your sense of self and interactions with others. 

You've mentioned that you feel unobservant and impulsive, which can be unsettling. We can work on mindfulness exercises that can help you become more aware of your surroundings and reactions to situations. 

I understand that you're feeling guilty about not doing more or being a better person. Remember, it's okay to not have all the answers and it's okay to just be. You're not alone, and together, we can navigate these feelings and work towards improving your mental health. 

Lastly, about your interests, it's okay to not have many interests or hobbies. However, exploring new ones could be a great way for you to connect with others and yourself. Let's start by considering things you've always wanted to do or try, and take it from there.

Remember, it's okay to reach out and ask for help. You're not alone in this journey.
'''

In [ ]:
mindfulness_therapy = '''
I understand that you're feeling lost, and it's okay to feel this way. It's important to remember that everyone has their own unique journey and it's perfectly okay to not have everything figured out.

You mentioned feeling scared and judgmental of yourself socially, and that you crave approval. It's not narcissistic to desire approval, it's human. What we'll work on is helping you find a sense of self-approval and self-love, so that you're less reliant on external validation.

You also expressed a sense of disconnection from your emotions. Through mindfulness exercises, we'll work on helping you reconnect with your feelings. Mindfulness can help you become an observer of your emotions, rather than feeling overwhelmed or disconnected from them.

Regarding your interests, it's okay to not have a long list. We can explore together some activities that might bring you joy and satisfaction. Remember, it's not about doing what everyone else is doing, it's about finding what brings you happiness.

In terms of relationships, it's important to remember that it's okay to be single. A partner can be a wonderful addition to life, but it's equally important to feel confident and happy on your own. We can work on developing healthy expectations and boundaries in relationships.

Lastly, you mentioned feeling unobservant and impulsive. We'll explore cognitive behavioral techniques to help you challenge these patterns of thinking and behaving, and develop more effective strategies for dealing with real-world scenarios. 

Remember, it's okay to feel the way you do. You're not alone, and there is help available for you. It's a brave step to reach out, and I encourage you to keep going. You're much stronger than you think.
'''

In [ ]:
counsellor_advices = [cognitive_response, interpersonal_therapy, mindfulness_therapy]

In [ ]:
counsellor_advice_choices = ""
for index, advice in enumerate(counsellor_advices):
    counsellor_advice_choices += "Advice " + str(index+1) + ":\n\n" + advice + "\n\n"

In [ ]:
final_prompt_summarize = user_summarize_prompt.format(user_input=x, user_past_history=sdoh_prompt, counselling_advices=counsellor_advice_choices)

In [ ]:
print(final_prompt_summarize)

In [ ]:
output = gpt(user_prompt=final_prompt_summarize, system_prompt=system_summarize_prompt, temperature=0.7, n=1)

In [ ]:
print(output[0])